In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import json
import re
import requests
import time

import swagger_client
from swagger_client.rest import ApiException

In [2]:
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.DEFAULT_SSL_CIPHER_LIST += 'HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

In [3]:
load_dotenv()
AEMET_KEY = os.getenv("KEY_AEMET")
configuration = swagger_client.Configuration()
configuration.api_key['api_key'] = AEMET_KEY
api_instance = swagger_client.ValoresClimatologicosApi(swagger_client.ApiClient(configuration))

In [4]:
df15 = pd.read_csv('data/Temp15-16_meteo.csv')
df16 = pd.read_csv('data/Temp16-17_meteo.csv')
df17 = pd.read_csv('data/Temp17-18_meteo.csv')

In [9]:
df15.loc[(df15.presmin == '-')]

,Equipo Local,Goles,Ataque,Medio,Defensa,Goles_Acumulados,Equipo Visitante,Goles.1,Ataque.1,Medio.1,...,Resultado,cod_est,tmin,tmed,tmax,prec,presmin,presmax,velmedia,rachaviento


In [6]:
est_meteo_mod15 = {
'Sporting de Gijón': {'prov':'Asturias',
                      'cod_est':'1208H'},

'Deportivo de La Coruña': {'prov':'A Coruña',
                           'cod_est':'1387E'},
}

In [7]:
df15.loc[(df15.presmin == '-'), 'cod_est'] = df15[(df15.presmin == '-')]['Equipo Local'].apply(lambda x: est_meteo_mod15[x]['cod_est'])

In [8]:
for index, row in df15.loc[(df15.presmin == '-')].iterrows():
    print(f'Obteniendo datos de la fila: {index}')
    fecha_ini = row.Día + 'T00:00:00UTC'
    fecha_fin = row.Día + 'T23:59:59UTC'
    estacion = row.cod_est
    
    try:
        respuesta_api = api_instance.climatologas_diarias_(fecha_ini, fecha_fin, estacion)
        url_respuesta = respuesta_api.datos
        response = requests.request("GET", url_respuesta)
        print(response)
        print('\n\n')
        meteo = json.loads(response.text)[0]
        
        try:
            df15.loc[index, 'presmin'] = meteo['presMin']
        except:
            df15.loc[index, 'presmin'] = '-'
        try:
            df15.loc[index, 'presmax'] = meteo['presMax']
        except:
            df15.loc[index, 'presmax'] = '-'

        time.sleep(5)
    except ApiException as e:
        print ('Algo ha fallado')
    

Obteniendo datos de la fila: 98
<Response [200]>



Obteniendo datos de la fila: 115
<Response [200]>



Obteniendo datos de la fila: 137
<Response [200]>



Obteniendo datos de la fila: 159
<Response [200]>



Obteniendo datos de la fila: 179
<Response [200]>



Obteniendo datos de la fila: 224
<Response [200]>



Obteniendo datos de la fila: 230
<Response [200]>



Obteniendo datos de la fila: 252
<Response [200]>



Obteniendo datos de la fila: 332
<Response [200]>



Obteniendo datos de la fila: 377
<Response [200]>





In [12]:
df15.to_csv('data/Temp15-16_meteo.csv', index=False)

In [ ]:
est_meteo = {
'Eibar': {'prov':'Gipuzkoa',
         'cod_est':'1050J'},
'Granada': {'prov':'Granada',
            'cod_est':'5530E'},
'Sporting de Gijón': {'prov':'Asturias',
                      'cod_est':'1207U'},
'Girona': {'prov':'Girona',
           'cod_est':'0367'},
'Espanyol': {'prov':'Barcelona',
             'cod_est':'0076'},
'Rayo Vallecano': {'prov':'Madrid',
                   'cod_est':'3195'},
'Athletic Club': {'prov':'Bizkaia',
                  'cod_est':'1082'},
'Alavés': {'prov':'Araba',
           'cod_est':'9091R'},
'Osasuna': {'prov':'Navarra',
            'cod_est':'9263D'},
'Deportivo de La Coruña': {'prov':'A Coruña',
                           'cod_est':'1387'},
'Sevilla': {'prov':'Sevilla',
            'cod_est':'5783'},
'Real Madrid': {'prov':'Madrid',
                'cod_est':'3195'},
'Leganés': {'prov':'Madrid',
            'cod_est':'3195'},
'Levante': {'prov':'Valencia',
            'cod_est':'8414A'},
'Barcelona': {'prov':'Barcelona',
              'cod_est':'0076'},
'Getafe': {'prov':'Madrid',
           'cod_est':'3195'},
'Málaga': {'prov':'Málaga',
           'cod_est':'6156X'},
'Villarreal': {'prov':'Castellón',
               'cod_est':'8500A'},
'Atlético de Madrid': {'prov':'Madrid',
                       'cod_est':'3195'},
'Real Sociedad': {'prov':'Gipuzkoa',
                  'cod_est':'1014A'},
'Las Palmas': {'prov':'Las Palmas',
               'cod_est':'C659M'},
'Celta de Vigo': {'prov':'Pontevedra',
         'cod_est':'1495'},
'Betis': {'prov':'Sevilla',
          'cod_est':'5783'},
'Valencia': {'prov':'Valencia',
             'cod_est':'8414A'}

}
